In [198]:
# Importing Libraries
import ast
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt 
import seaborn as sns

# Loading Data
dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()

# Data Cleanup
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])
df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)


In [ ]:
df.head(3)

In [ ]:
df_DA_US = df[(df['job_title_short']=='Data Analyst') & (df['job_country']=='United States')].copy()

df_DA_US['job_posted_mo'] = df_DA_US['job_posted_date'].dt.month      #.to_period('M')
df_DA_US_explode = df_DA_US.explode('job_skills')
df_DA_US_explode.head(3)

In [ ]:
df_DA_US_pivot = pd.pivot_table(df_DA_US_explode,index='job_posted_mo',columns='job_skills',aggfunc='size')
df_DA_US_pivot.loc['total'] = df_DA_US_pivot.sum()
df_DA_US_pivot = df_DA_US_pivot[df_DA_US_pivot.loc['total'].sort_values(ascending=False).index]
df_DA_US_pivot = df_DA_US_pivot.drop(index=['total'])
# df_DA_US_pivot.columns
df_DA_US_pivot

In [ ]:
df_DA_US_pivot.iloc[:,:5].plot(kind='line')

plt.title('top 5 skills for data analysts, indexed by month')
plt.xlabel('')
plt.ylabel('number of occurances')
# plt.legend()
plt.show()

In [ ]:
df_plot = df_DA_US_pivot.iloc[:,:5]

for i,skill in enumerate(df_plot.columns):
    plt.plot(df_plot.index,df_plot[skill],label=skill)

plt.title('top 5 skills for data analysts, indexed by month')
plt.xlabel('')
plt.ylabel('number of occurances')
plt.legend()
plt.show()

In [ ]:
DA_totals = df_DA_US.groupby('job_posted_mo').size().reset_index(name='job_total')
DA_totals.set_index('job_posted_mo',inplace=True)
df_merge = pd.merge(df_DA_US_pivot,DA_totals,how='left',on='job_posted_mo')
df_merge.columns

df_merge_perc = {}

for i,skill in enumerate(df_merge.columns):
    skill_new = skill + '%'
    df_merge_perc[skill_new] = df_merge[skill]/df_merge['job_total']

df_merge_perc = pd.DataFrame(df_merge_perc)
df_merge_perc

In [ ]:
DA_totals = df_DA_US.groupby('job_posted_mo').size()
df_DA_US_pivot_percent = df_DA_US_pivot.iloc[:12].div(DA_totals/100,axis=0)
df_DA_US_pivot_percent = df_DA_US_pivot_percent.reset_index()
df_DA_US_pivot_percent['job_posted_month'] = df_DA_US_pivot_percent['job_posted_mo'].apply(lambda x: pd.to_datetime(x,format='%m').strftime('%b'))
df_DA_US_pivot_percent.set_index('job_posted_month',inplace=True)
df_DA_US_pivot_percent.drop(columns='job_posted_mo',inplace=True)
df_DA_US_pivot_percent
# type(DA_totals)

In [210]:
df_plot = df_DA_US_pivot_percent.iloc[:,:5]
# df_plot.plot(kind='line',marker='o')

# plt.xlabel('')
# plt.ylabel('%')
# plt.title('% per each job skill across one year')
# plt.legend(bbox_)

# plt.show()

In [213]:
df_plot.index[0]

'Jan'

In [ ]:
# df_DA_US_percent = df_DA_US_pivot.iloc[:12].div(DA_totals/100, axis=0)
df_DA_US_pivot.iloc[:12]

In [63]:
d = {'a': 1, 'b': 2, 'c': 3}
ser = pd.Series(data=d, index=['a', 'b', 'c'])
orig = ser.to_frame()
origT = orig.T
orig, origT


(   0
 a  1
 b  2
 c  3,
    a  b  c
 0  1  2  3)